# 📡 Notebook 1: API Setup & Authentication

## วัตถุประสงค์
สอนนักเรียนให้เชื่อมต่อกับ GRVT Exchange ได้อย่างถูกต้องและปลอดภัย

## สิ่งที่จะได้เรียนรู้
1.  GRVT Exchange คืออะไร (Hybrid Exchange)
2.  การสร้าง API Key
3.  ทำไม GRVT ใช้ Private Key แทน API Secret
4.  การตั้งค่า `X-Grvt-Account-Id` Header
5.  การทดสอบ Authentication กับ Public API
6.  การทดสอบ Private API (ดึงข้อมูล Account)

---

## 📚 Section 1: GRVT Exchange คืออะไร?

### Hybrid Exchange

**GRVT** เป็น **Hybrid Centralized-Decentralized Exchange** ที่รวมข้อดีของทั้งสองโลก:

| ลักษณะ | CEX (Centralized) | DEX (Decentralized) | GRVT (Hybrid) |
|--------|-------------------|---------------------|---------------|
| ความเร็ว |  เร็วมาก |  ช้า |  เร็วมาก |
| ค่าธรรมเนียม |  ปานกลาง |  สูง (Gas Fees) |  ต่ำ |
| ความปลอดภัย |  ต้องไว้ใจ Exchange |  ไม่ต้องไว้ใจ |  ไม่ต้องไว้ใจ |
| Self-Custody |  ไม่ได้ |  ได้ |  ได้ |

### จุดเด่นของ GRVT:
-  **Self-Custody**: คุณเป็นเจ้าของ private key ของคุณเอง
-  **High Performance**: ประมวลผลได้เร็วเหมือน CEX
-  **EIP-712 Signing**: ทุก order ต้องลงนามด้วย private key (เรียนรู้ใน Notebook 2)
-  **Smart Contract Protection**: มีการพิสูจน์บน blockchain

---

## ⚙️ Section 2: การตั้งค่าเบื้องต้น

### ขั้นตอนที่ 1: Import Libraries

In [ ]:
# Import standard libraries
import os
import sys
import logging
from pprint import pprint

# Import helper functions
from grvt_helpers import (
    load_grvt_config,
    validate_config,
    connect_to_grvt,
    print_section_header,
    run_preflight_checks
)

print(" Import สำเร็จ!")

### ขั้นตอนที่ 2: โหลด Configuration

 **สำคัญ**: ต้องมีไฟล์ `.env` ในโฟลเดอร์เดียวกับ notebook นี้

ถ้ายังไม่มี ให้:
1. คัดลอก `.env.template` เป็น `.env`
2. กรอก API credentials ของคุณ

In [ ]:
# โหลด configuration จาก .env file
config = load_grvt_config()

# แสดงข้อมูล (ซ่อน sensitive data)
print(" Configuration Loaded:")
print(f"  Environment: {config['GRVT_ENV']}")
print(f"  API Key: {config['GRVT_API_KEY'][:10]}..." if config['GRVT_API_KEY'] else "  API Key: Not Set")
print(f"  Trading Account: {config['GRVT_TRADING_ACCOUNT_ID'][:10]}..." if config['GRVT_TRADING_ACCOUNT_ID'] else "  Trading Account: Not Set")
print(f"  Sub Account ID: {config['GRVT_SUB_ACCOUNT_ID']}")

### ขั้นตอนที่ 3: ตรวจสอบ Configuration

ตรวจสอบว่ากรอกข้อมูลครบถ้วนหรือไม่

In [ ]:
# ตรวจสอบว่า config ครบถ้วน
is_valid, missing_fields = validate_config(config)

if is_valid:
    print(" Configuration ครบถ้วน!")
else:
    print(" Configuration ไม่ครบ!")
    print(f"   ขาดฟิลด์: {', '.join(missing_fields)}")
    print("\n กรุณาแก้ไขไฟล์ .env แล้วรัน cell นี้อีกครั้ง")

---

## 🔌 Section 3: เชื่อมต่อกับ GRVT API

### ทำความเข้าใจกับ Authentication Flow

```mermaid
sequenceDiagram
    participant Bot as Trading Bot
    participant API as GRVT API
    participant BC as Blockchain
    
    Bot->>API: 1. ส่ง API Key
    API->>API: 2. Validate API Key
    API-->>Bot: 3. Return Session Cookie
    
    Bot->>API: 4. Request (with Cookie + X-Grvt-Account-Id)
    API->>API: 5. Verify Session
    API-->>Bot: 6. Return Data
    
    Note over Bot,BC: Orders ต้องลงนามด้วย Private Key (EIP-712)
```

###  GRVT Credentials (ต่างจาก Exchange ทั่วไป!)

GRVT เป็น **Hybrid Exchange** จึงต้องการ credentials ที่แตกต่าง:

#### Exchange ทั่วไป (Binance, Bybit):
```
 API_KEY
 API_SECRET  ← sign requests
```

#### GRVT (Hybrid Exchange):
```
 GRVT_API_KEY        ← authenticate
 GRVT_API_SECRET     ← ไม่มี!
 GRVT_PRIVATE_KEY    ← sign orders (EIP-712)
 GRVT_TRADING_ACCOUNT_ID
```

### รายละเอียดแต่ละ Credential:

1. **GRVT_API_KEY** 
   - ใช้สำหรับ authenticate กับ GRVT API
   - ระบุตัวตนว่าคุณคือใคร
   
2. **GRVT_PRIVATE_KEY** (Ethereum Private Key) 
   - **ใช้แทน API_SECRET!**
   - ลงนาม (sign) orders ด้วย **EIP-712**
   - พิสูจน์ความเป็นเจ้าของ on-chain
   - เรียนรู้ EIP-712 ในรายละเอียดใน Notebook 2
   
3. **GRVT_TRADING_ACCOUNT_ID** 
   - บัญชีซื้อขายของคุณบน GRVT

>  **สำคัญ**: GRVT **ไม่มี** `API_SECRET`!  
> ใช้ `PRIVATE_KEY` (Ethereum) สำหรับ signing orders แทน  
> อ่านเพิ่มเติม: [CREDENTIALS_GUIDE.md](./CREDENTIALS_GUIDE.md)

### ขั้นตอนที่ 4: สร้าง Connection

In [ ]:
# สร้าง logger สำหรับ debug
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('GRVT')

# เชื่อมต่อกับ GRVT
print_section_header("🔌 เชื่อมต่อกับ GRVT")
client = connect_to_grvt(config, logger)

if client:
    print("\n เชื่อมต่อสำเร็จ!")
    print(f"   Environment: {config['GRVT_ENV']}")
else:
    print("\n เชื่อมต่อไม่สำเร็จ")
    print("   กรุณาตรวจสอบ credentials ใน .env file")

---

##  Section 4: ทดสอบ Public API

**Public API** = API ที่ไม่ต้อง authentication ก็เรียกได้  
เช่น: ดูราคา, ดู order book, ดู trades ล่าสุด

### ทดสอบดึงราคา BTC

In [ ]:
# Helper function to safely convert to float
def safe_float(value, default=0.0):
    """Convert value to float, return default if conversion fails"""
    if value is None or value == 'N/A' or value == '':
        return default
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

# ดึงราคาตลาดปัจจุบัน (Public API - ไม่ต้อง authentication)
symbol = 'BTC_USDT_Perp'

print_section_header(f" ดึงราคา {symbol}")

try:
    ticker = client.fetch_ticker(symbol)
    
    # GRVT API ใช้ชื่อ field ที่แตกต่างจาก CCXT standard
    # - best_bid_price แทน bid
    # - best_ask_price แทน ask
    # - buy_volume_24h_b + sell_volume_24h_b แทน volume
    
    # ดึงข้อมูลจาก GRVT specific fields
    last_price = safe_float(ticker.get('last_price', ticker.get('last')))
    mark_price = safe_float(ticker.get('mark_price'))
    index_price = safe_float(ticker.get('index_price'))
    
    # Bid/Ask prices
    bid = safe_float(ticker.get('best_bid_price', ticker.get('bid')))
    ask = safe_float(ticker.get('best_ask_price', ticker.get('ask')))
    
    # Volume (รวม buy + sell)
    buy_vol = safe_float(ticker.get('buy_volume_24h_b'))
    sell_vol = safe_float(ticker.get('sell_volume_24h_b'))
    total_volume = buy_vol + sell_vol
    
    # 24h high/low
    high_24h = safe_float(ticker.get('high_price'))
    low_24h = safe_float(ticker.get('low_price'))
    
    print(f" {symbol}")
    print(f"\n💰 Prices:")
    print(f"  Last Price:  ${last_price:,.2f}")
    print(f"  Mark Price:  ${mark_price:,.2f}")
    print(f"  Index Price: ${index_price:,.2f}")
    
    print(f"\n Order Book:")
    print(f"  Best Bid: ${bid:,.2f}")
    print(f"  Best Ask: ${ask:,.2f}")
    
    if bid > 0 and ask > 0:
        spread = ask - bid
        spread_pct = (spread / bid) * 100
        print(f"  Spread:   ${spread:.2f} ({spread_pct:.3f}%)")
    
    print(f"\n 24h Stats:")
    print(f"  Volume:  {total_volume:,.3f} BTC (Buy: {buy_vol:.3f}, Sell: {sell_vol:.3f})")
    print(f"  High:    ${high_24h:,.2f}")
    print(f"  Low:     ${low_24h:,.2f}")
    
    # if high_24h > 0 and low_24h > 0:
    #     change_24h = ((last_price - ticker.get('open_price', last_price)) / ticker.get('open_price', last_price)) * 100
    #     print(f"  Change:  {change_24h:+.2f}%")
    
    print("\n Public API ทำงานได้ปกติ!")
    
except Exception as e:
    print(f" Error: {e}")
    print("\n ตรวจสอบว่า:")
    print("   1. Internet connection ทำงานปกติ")
    print("   2. Symbol ถูกต้อง")
    print("   3. GRVT API service ทำงานปกติ")

2026-01-27 16:33:42,514 - GRVT - INFO - cookie should be refreshed self._cookie=None now=1769506422.5140517 time_till_expiration=None secs



  💰 ดึงราคา BTC_USDT_Perp



2026-01-27 16:33:43,321 - root - ERROR - get_cookie_with_expiration path='https://edge.testnet.grvt.io/auth/api_key/login' Error getting cookie: 'gravity'
2026-01-27 16:33:43,323 - GRVT - INFO - _auth_and_post path='https://market-data.testnet.grvt.io/full/v1/ticker' payload={'instrument': 'BTC_USDT_Perp'}
payload_json='{"instrument": "BTC_USDT_Perp"}'
2026-01-27 16:33:43,449 - GRVT - INFO - _auth_and_post path='https://market-data.testnet.grvt.io/full/v1/ticker' OK return_value=<Response [200]> response={'result': {'event_time': '1769506423698337187', 'instrument': 'BTC_USDT_Perp', 'mark_price': '87941.30769405', 'index_price': '87962.215008802', 'last_price': '88050.0', 'last_size': '0.1', 'mid_price': '87949.5', 'best_bid_price': '87948.0', 'best_bid_size': '0.428', 'best_ask_price': '87951.0', 'best_ask_size': '2.249', 'funding_rate_8h_curr': '0.01', 'funding_rate_8h_avg': '0.01', 'interest_rate': '0.0', 'forward_price': '0.0', 'buy_volume_24h_b': '214.938', 'sell_volume_24h_b': '1

📊 BTC_USDT_Perp

💰 Prices:
  Last Price:  $88,050.00
  Mark Price:  $87,941.31
  Index Price: $87,962.22

📈 Order Book:
  Best Bid: $87,948.00
  Best Ask: $87,951.00
  Spread:   $3.00 (0.003%)

📊 24h Stats:
  Volume:  333.570 BTC (Buy: 214.938, Sell: 118.632)
  High:    $88,802.00
  Low:     $87,253.00

✅ Public API ทำงานได้ปกติ!


### ดูข้อมูลเพิ่มเติมของ Ticker

In [ ]:
# แสดงข้อมูลทั้งหมดของ ticker
print(" ข้อมูลทั้งหมดของ Ticker:")
print("-" * 60)
pprint(ticker)

📊 ข้อมูลทั้งหมดของ Ticker:
------------------------------------------------------------
{'best_ask_price': '87951.0',
 'best_ask_size': '2.249',
 'best_bid_price': '87948.0',
 'best_bid_size': '0.428',
 'buy_volume_24h_b': '214.938',
 'buy_volume_24h_q': '18895862.6885',
 'event_time': '1769506423698337187',
 'forward_price': '0.0',
 'funding_rate': '0.01',
 'funding_rate_8h_avg': '0.01',
 'funding_rate_8h_curr': '0.01',
 'high_price': '88802.0',
 'index_price': '87962.215008802',
 'instrument': 'BTC_USDT_Perp',
 'interest_rate': '0.0',
 'last_price': '88050.0',
 'last_size': '0.1',
 'long_short_ratio': '0.817680',
 'low_price': '87253.0',
 'mark_price': '87941.30769405',
 'mid_price': '87949.5',
 'next_funding_time': '1769529600000000000',
 'open_interest': '6234.204',
 'open_price': '87780.0',
 'sell_volume_24h_b': '118.632',
 'sell_volume_24h_q': '10451794.23'}


---

##  Section 5: ทดสอบ Private API

**Private API** = API ที่ต้อง authentication  
เช่น: ดู balance, ดู positions, วาง orders, cancel orders

### สิ่งสำคัญ: X-Grvt-Account-Id Header

 **ทุกครั้ง** ที่เรียก Private API ของ GRVT ต้องใส่:
```
X-Grvt-Account-Id: <your_trading_account_id>
```

SDK จะทำส่วนนี้ให้อัตโนมัติ แต่ต้องรู้ไว้เผื่อใช้ raw HTTP requests

### ทดสอบดึงข้อมูล Account Balance

In [ ]:
print_section_header(" ดึงข้อมูล Account Balance")

try:
    # ดึงข้อมูล balance (Private API)
    balance = client.fetch_balance()
    
    print(" Authentication สำเร็จ!\n")
    
    # แสดงข้อมูล balance หลักๆ
    if 'total' in balance:
        print(" Account Summary:")
        total_equity = safe_float(balance.get('total', {}).get('USDT'))
        print(f"  Total Equity: ${total_equity:,.2f}")
    
    if 'free' in balance:
        available = safe_float(balance.get('free', {}).get('USDT'))
        print(f"  Available Balance: ${available:,.2f}")
    
    if 'used' in balance:
        used_margin = safe_float(balance.get('used', {}).get('USDT'))
        print(f"  Used Margin: ${used_margin:,.2f}")
    
except Exception as e:
    print(f" Error: {e}")
    print("\n ตรวจสอบว่า:")
    print("   1. API Key ถูกต้อง")
    print("   2. Trading Account ID ถูกต้อง")
    print("   3. API Key มี permission ในการดู balance")

2026-01-27 16:38:07,438 - GRVT - INFO - cookie should be refreshed self._cookie=None now=1769506687.4385033 time_till_expiration=None secs



  💰 ดึงข้อมูล Account Balance



2026-01-27 16:38:08,296 - root - ERROR - get_cookie_with_expiration path='https://edge.testnet.grvt.io/auth/api_key/login' Error getting cookie: 'gravity'
2026-01-27 16:38:08,298 - GRVT - INFO - _auth_and_post path='https://trades.testnet.grvt.io/full/v1/account_summary' payload={'sub_account_id': 'your_account_id_here'}
payload_json='{"sub_account_id": "your_account_id_here"}'
2026-01-27 16:38:08,567 - GRVT - WARNING - _auth_and_post path='https://trades.testnet.grvt.io/full/v1/account_summary' ERROR payload_json='{"sub_account_id": "your_account_id_here"}'
return_value=<Response [401]>
response={'code': 1000, 'message': 'You need to authenticate prior to using this functionality', 'status': 401}
2026-01-27 16:38:08,567 - GRVT - INFO - GrvtCcxt get_account_summary type='sub-account' No account summary for path='https://trades.testnet.grvt.io/full/v1/account_summary' payload={'sub_account_id': 'your_account_id_here'}


✅ Authentication สำเร็จ!

📊 Account Summary:
  Total Equity: $0.00
  Available Balance: $0.00
  Used Margin: $0.00


### ดูข้อมูลทั้งหมดของ Balance

In [ ]:
# แสดงข้อมูลทั้งหมดของ balance
print(" ข้อมูลทั้งหมดของ Balance:")
print("-" * 60)
pprint(balance)

---

## 🧪 Section 6: Pre-flight Checks

รัน automated checks เพื่อตรวจสอบว่าทุกอย่างพร้อมใช้งาน

In [ ]:
# รัน pre-flight checks
results = run_preflight_checks(config)

# สรุปผลการตรวจสอบ
print("\n" + "="*60)
print(" Checklist Summary:")
print("="*60)

for check_name, passed in results.items():
    status = "" if passed else "❌"
    print(f"{status} {check_name}: {'PASSED' if passed else 'FAILED'}")

# Overall result
all_passed = all(results.values())
print("\n" + "="*60)
if all_passed:
    print(" ALL CHECKS PASSED - พร้อมใช้งาน!")
else:
    print(" SOME CHECKS FAILED - กรุณาแก้ไขปัญหาก่อนใช้งาน")
print("="*60)

---

##  สรุป

ใน Notebook นี้เราได้เรียนรู้:

1.  **GRVT คือ Hybrid Exchange** ที่รวมข้อดีของ CEX และ DEX
2.  **GRVT ใช้ PRIVATE_KEY แทน API_SECRET** สำหรับ signing orders
3.  **Configuration** โหลดจาก `.env` file เพื่อความปลอดภัย
4.  **Public API** ใช้ได้โดยไม่ต้อง authentication
5.  **GRVT API ใช้ field names ที่เป็นเอกลักษณ์** (เช่น best_bid_price, best_ask_price)
6.  **Private API** ต้องใช้ API Key + Trading Account ID
7.  **X-Grvt-Account-Id Header** ต้องใส่ทุกครั้งสำหรับ Private API
8.  **Pre-flight Checks** ช่วยตรวจสอบว่าทุกอย่างพร้อมใช้งาน

###  ขั้นตอนต่อไป

ไปที่ **Notebook 2: EIP-712 Signing** เพื่อเรียนรู้:
- EIP-712 คืออะไร
- ทำไม GRVT ต้องใช้ EIP-712 แทน API Secret
- วิธีการลงนาม Orders ด้วย Private Key
- การ verify signatures

---

##  ข้อควรระวัง

> **Security Best Practices:**
> 
> 1.  **อย่า commit** ไฟล์ `.env` เข้า git
> 2.  **อย่าแชร์** API Key หรือ Private Key
> 3.  **ใช้ Testnet** ก่อนเสมอสำหรับการทดสอบ
> 4.  **จำกัด permissions** ของ API Key ให้เหมาะสม
> 5.  **Rotate keys** เป็นประจำ

---

##  Resources

- [GRVT Official Documentation](https://docs.grvt.io/)
- [API Reference](https://docs.grvt.io/api)
- [Testnet](https://testnet.grvt.io/)
- [CREDENTIALS_GUIDE.md](./CREDENTIALS_GUIDE.md) - คำอธิบายโดยละเอียดเกี่ยวกับ credentials